<a href="https://colab.research.google.com/github/tantowijh/colab-collection/blob/main/FFMPEG_CONVERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <img src='https://forum.rclone.org/uploads/default/original/2X/d/da6ccb2784ff3fa73d9339300530e0aae4d6cebd.png' height="45" alt="rclone"/>

In [7]:
# ============================= FORM ============================= #
#@title <font size="4"><b>Step 1. </b>Upload your <code>rclone.conf</code> profile</font> { vertical-output: true }
#@markdown <h6>*<i>Required to mount your drive!</i></h4>
## @markdown <h7>Your <code>rclone.conf</code> file is located at:</i></h4>
# @markdown ### <font size=1px><i>For not able to upload local file : https://stackoverflow.com/a/58661947</font>
# ================================================================ #
MODE = "RCONFIG"
from os import path as _p
from IPython.display import clear_output

if not _p.exists("/root/.ipython/rlab_utils.py"):
    from shlex import split as _spl
    from subprocess import run  # nosec

    shellCmd = "wget -qq https://raw.githubusercontent.com/biplobsd/RLabClone/master/res/rlab_utils.py \
                    -O /root/.ipython/rlab_utils.py"
    run(_spl(shellCmd))  # nosec

import importlib, rlab_utils
from google.colab import files  # pylint: disable=import-error #nosec
from rlab_utils import checkAvailable, runSh, PATH_RClone_Config, prepareSession


def generateUploadList():
    prepareSession()
    if checkAvailable("/content/upload.txt"):
        runSh("rm -f upload.txt")
    runSh(
        f"rclone --config {PATH_RClone_Config}/rclone.conf lsf {REMOTE}: --include '{QUERY_PATTERN}' --drive-shared-with-me --files-only --max-depth 1 > /content/upload.txt",
        shell=True,  # nosec
    )


def uploadLocalFiles():
    prepareSession()
    if MODE == "UTILS":
        filePath = "/root/.ipython/rlab_utils.py"
    elif MODE in ("RCONFIG", "RCONFIG_append"):
        filePath = f"{PATH_RClone_Config}/rclone.conf"
    else:
        pass

    try:
      if checkAvailable(filePath):
        runSh(f"rm -f {filePath}")
      print("Select file from your computer.\n")
      uploadedFile = files.upload()
      fileNameDictKeys = uploadedFile.keys()
      fileNo = len(fileNameDictKeys)
      if fileNo > 1:
          for fn in fileNameDictKeys:
              runSh(f'rm -f "/content/{fn}"')
          return print("\nPlease only upload a single config file.")
      elif fileNo == 0:
          return print("\nFile upload cancelled.")
      elif fileNo == 1:
          for fn in fileNameDictKeys:
              if checkAvailable(f"/content/{fn}"):
                  if MODE == "RCONFIG_append":
                    import urllib
                    urllib.request.urlretrieve("https://biplobsd.github.io/RLabClone/res/rclonelab/rclone.conf",
                                               "/usr/local/sessionSettings/rclone.conf")
                    with open(f"/content/{fn}", 'r+') as r:
                      new_data = r.read()
                      runSh(f'rm -f "/content/{fn}"')
                    with open(filePath, 'r+') as f:
                      old_data = f.read()
                      f.seek(0)
                      f.truncate(0)
                      f.write(old_data + new_data)
                    print("\nUpdate completed.")
                  else:
                    runSh(f'mv -f "/content/{fn}" {filePath}')
                    runSh(f"chmod 666 {filePath}")
                    runSh(f'rm -f "/content/{fn}"')
                    importlib.reload(rlab_utils)
                    print("Upload completed.")
              return
      else:
          print("\nNo file")
          return
    except:
      return print("\nUpload process Error.")


if MODE == "GENERATELIST":
    generateUploadList()
else:
    uploadLocalFiles()


Select file from your computer.



Saving rclone.conf to rclone.conf
Upload completed.


In [16]:
# ============================= FORM ============================= #
#@title <font size="4"><b>Step 2</b>. Mount your Drive with rclone</font> { vertical-output: true }
Cache_Directory = "DISK"
import os
from IPython.display import HTML, clear_output
import uuid
import ipywidgets as widgets
from google.colab import output
import re
#####################################
if not os.path.exists("/root/.ipython/rlab_utils.py"):
  from shlex import split as _spl
  from subprocess import run

  shellCmd = "wget -qq https://biplobsd.github.io/RLabClone/res/rlab_utils.py \
                  -O /root/.ipython/rlab_utils.py"
  run(_spl(shellCmd))

from rlab_utils import (
    runSh,
    prepareSession,
    PATH_RClone_Config,
)

class MakeButton(object):
  def __init__(self, title, callback, style):
    self._title = title
    self._callback = callback
    self._style = style
  def _repr_html_(self):
    callback_id = 'button-' + str(uuid.uuid4())
    output.register_callback(callback_id, self._callback)
    if self._style != "":
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button mod-" + self._style
    else:
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button"
    template = """<button class="{style_html}" id="{callback_id}">{title}</button>
        <script>
          document.querySelector("#{callback_id}").onclick = (e) => {{
            google.colab.kernel.invokeFunction('{callback_id}', [], {{}})
            e.preventDefault();
          }};
        </script>"""
    html = template.format(title=self._title, callback_id=callback_id, style_html=style_html)
    return html
  
def ShowAC():
  clear_output(wait=True)
  display(
      widgets.HBox(
          [widgets.VBox(
              [widgets.HTML(
                  '''<h3 style="font-family:Trebuchet MS;color:#127ACC;margin-top:0px;">
                  Choose Drive to Mount:</h3>
                  '''
                  ),
               mountNam]
               )
          ]
          )
      )
  
  display(HTML("<br>"), MakeButton("Mount", MountCMD, "primary"),
          MakeButton("Unmount", unmountCMD, "danger"))

prepareSession()
content = open(f"{PATH_RClone_Config}/rclone.conf").read()
avCon = re.findall(r"^\[(.+)\]$", content, re.M)
mountNam = widgets.Dropdown(options=avCon)

if Cache_Directory == 'RAM':
  cache_path = '/dev/shm'
elif Cache_Directory == 'DISK':
  os.makedirs('/tmp', exist_ok=True)
  cache_path = '/tmp'

def MountCMD():
    mPoint = f"/content/drives/{mountNam.value}"
    os.makedirs(mPoint, exist_ok=True)
    cmd = rf"rclone mount {mountNam.value}: {mPoint}" \
      rf" --config {PATH_RClone_Config}/rclone.conf" \
      ' --daemon'

    if runSh(cmd, shell=True) == 0:
      print(f"Mount success! - \t{mPoint}")
    else:
      print(f"Mount failed! - \t{mPoint}")

def unmountCMD():
  mPoint = f"/content/drives/{mountNam.value}"
  if os.system(f"fusermount -uz {mPoint}") == 0:
    runSh(f"rm -r {mPoint}")
    print(f"Unmounted success! - \t{mPoint}")
  else:
    runSh(f"fusermount -uz {mPoint}", output=True)

ShowAC()

Unmounted success! - 	/content/drives/AnimeServer
Mount success! - 	/content/drives/AnimeServer


# <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/d/da/Google_Drive_logo.png/600px-Google_Drive_logo.png" height=60px> <img src="https://www.freeiconspng.com/thumbs/font-icon/font-icon-9.png" height=70px>

In [ ]:
#@markdown <br><center><img src='https://upload.wikimedia.org/wikipedia/commons/thumb/d/da/Google_Drive_logo.png/600px-Google_Drive_logo.png' height="50" alt="Gdrive-logo"/></center>
#@markdown <center><h3>Mount Gdrive to /content/drive</h3></center><br>
MODE = "MOUNT" #@param ["MOUNT", "UNMOUNT"]
#Mount your Gdrive! 
from google.colab import drive
drive.mount._DEBUG = False
if MODE == "MOUNT":
  drive.mount('/content/drive', force_remount=True)
elif MODE == "UNMOUNT":
  try:
    drive.flush_and_unmount()
  except ValueError:
    pass
  get_ipython().system_raw("rm -rf /root/.config/Google/DriveFS")

In [ ]:
#@title **Runtime Prolonger**
#@markdown *This cell runs a JS code that automatically reconnects you to avoid inactivity due to idle.*

import IPython
from IPython.display import clear_output
from google.colab import output

display(IPython.display.Javascript('''
 function connectRefresher() {
       window.ConnectButtonIntervalId = setInterval(function ConnectButton(){
                console.log("connected"); 
                document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click();
                document.querySelector("colab-sessions-dialog").shadowRoot.querySelector("#footer > div > paper-button").click();
                console.log("closed the dialog!!"); 
            },60000);
    }
    
 function clearRefresher() { 
           console.log("clear Interval called !!");
           clearInterval(window.ConnectButtonIntervalId);
    }

 connectRefresher(); //to connect the refresher
 clearRefresher(); //to disconnect the refresher
'''))

clear_output()

In [11]:
#@title **Font Installer**
#@markdown *This cell runs a code that automatically add your custom-fonts. Please add your custom-fonts folder path location here!*

FontsLinks = "https://github.com/tantowijh/Fonts-Anime" #@param {type:"string"}
FontsDestination = "/usr/share/fonts/truetype/Anime"

!git clone '{FontsLinks}' '{FontsDestination}'
#!sudo mv -v -r *.* '{FontsDestination}'

import IPython
from IPython.display import clear_output
from google.colab import output

clear_output()

In [ ]:
#@title **Google Fonts**
#@markdown *This cell runs a code that automatically install entire google-fonts library!*

import IPython
from IPython.display import clear_output
from google.colab import output

!git clone https://github.com/google/fonts.git
%mv /content/fonts/ofl/* /usr/share/fonts/truetype
%cd /content
%rm -rf "fonts"

clear_output()

# <img src='https://www.vozidea.com/wp-content/uploads/2018/05/ffmpeg-logo-grande.png ' height="57" alt="RcloneLab"/>

In [9]:
#@markdown <h3>⬅️ Click Here to Install FFmpeg, VCSI & Mkvtoolnix</h3>

!apt install ffmpeg
!sudo apt-get install mkvtoolnix 
!pip install vcsi


from IPython.display import HTML, clear_output
clear_output()
print("Server Started Successfully")

Server Started Successfully


In [10]:
#@markdown <h3>⬅️ Click Here to Check GPU</h3>
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-2e784939-8fa3-af9b-d664-8d1cf94e149a)



###» **Video to (*H264*-AAC 224K)  - Need GPU - Nvidia Telsa P100 or T4**

####» Single Video Encoder

In [ ]:
#@markdown <h3><b>Single File Encode :</h3>
#@markdown <i>Put file name (include the extension). Convert single video in directory.</i>

import os, sys, re

InputName = '/content/drive/Shareddrives/01. Anime Server/01. Anime Series/Osananajimi ga Zettai ni Makenai Love Comedy/Season 1/[LENDRIVE] Osananajimi ga Zettai ni Makenai Love Comedy - 10 [1080p] [H265] [Dualsubs].mkv' #@param {type:"string"}
OutputName = '/content/drive/Shareddrives/01. Anime Server/01. Anime Series/Osananajimi ga Zettai ni Makenai Love Comedy/Season 1/Streaming Ready/Osananajimi ga Zettai ni Makenai Love Comedy - 10 [1080p] [H265] [Dualsubs]' #@param {type:"string"}
OutputFormat = ".mp4" #@param [".mp4", ".mov", ".mkv", ".avi", ".ts", ".flv", ".webm", ".wmv", ".mpg", ".m4v", ".f4v", ".m2ts", ".mpeg", ".3gp", ".MP4", ".MOV", ".MKV", ".AVI", ".TS", ".FLV", ".WEBM", ".WMV", ".MPG", ".M4V", ".F4V", ".M2TS", ".MPEG", ".3GP"]

#@markdown <b><i>Subtitle Options.</i></b>
BurnIn = True #@param {type:"boolean"}
SubtitleTrack = True #@param {type:"boolean"}
TrackNumber = 0 #@param {type:"slider", min:0, max:10, step:1}

#@markdown <h3><b>Video Quality :</h3>
#@markdown <h><i>Between 0 [Best] to 51 [Worst]</i><h>
Set_cq = '23'#@param {type:"string"}
Set_qmin = '2'#@param {type:"string"}
Set_qmax = '31'#@param {type:"string"}

#!ffmpeg -hwaccel cuvid -i '{InputName}' -c:v h264_nvenc -pix_fmt yuv420p -preset slow -rc vbr_hq -b:v 8M -maxrate:v 10M -c:a aac -b:a 224k '{OutputName}'$OutputFormat

def noSubtitle():
  cmd = !ffmpeg -y -hwaccel cuvid -i '{InputName}' -c:v h264_nvenc -pix_fmt yuv420p -preset slow -rc vbr_hq -cq $Set_cq -qmin $Set_qmin -qmax $Set_qmax -b:v 8M -maxrate:v 10M -c:a aac -b:a 224k '{OutputName}'$OutputFormat
def burnSubtitle():
  def noTrack():
    cmd = !ffmpeg -y -hwaccel cuvid -i '{InputName}' -c:v h264_nvenc -pix_fmt yuv420p -preset slow -rc vbr_hq -cq $Set_cq -qmin $Set_qmin -qmax $Set_qmax -b:v 8M -maxrate:v 10M -c:a aac -b:a 224k -filter_complex "subtitles='{InputName}'" '{OutputName}'$OutputFormat
  def siTrack():
    cmd = !ffmpeg -y -hwaccel cuvid -i '{InputName}' -c:v h264_nvenc -pix_fmt yuv420p -preset slow -rc vbr_hq -cq $Set_cq -qmin $Set_qmin -qmax $Set_qmax -b:v 8M -maxrate:v 10M -c:a aac -b:a 224k -filter_complex "subtitles='{InputName}:si={TrackNumber}'" '{OutputName}'$OutputFormat
  if SubtitleTrack == False:
    noTrack()
  else:
    siTrack()
if BurnIn == False:
  noSubtitle()
else:
  burnSubtitle()

#!ffmpeg -y -hwaccel cuvid -i '{InputName}' -c:v h264_nvenc -pix_fmt yuv420p -preset slow -rc vbr_hq -cq $Set_cq -qmin $Set_qmin -qmax $Set_qmax -b:v 8M -maxrate:v 10M -c:a aac -b:a 224k -filter_complex "subtitles='{InputName}'" '{OutputName}'$OutputFormat

####» Full Directory Encoder

In [ ]:
#@markdown <h3><b>Full Directory Encode :</h3>
#@markdown <i>Put file name (include the extension). Convert the entire videos in directory.</i>

import os, sys, re
import subprocess

InputLocation = '/content/drives/AnimeServer/00. Simple-Manager/Fate Animes and Movies/02. Fate Stay Night/Fate Stay Night (Movie)' #@param {type:"string"}
OutputLocation = '/content/drives/AnimeServer/00. Simple-Manager/Fate Animes and Movies/02. Fate Stay Night/Fate Stay Night (Movie)/Streaming Ready' #@param {type:"string"}
InputFormat = ".mkv" #@param [".mp4", ".mov", ".mkv", ".avi", ".ts", ".flv", ".webm", ".wmv", ".mpg", ".m4v", ".f4v", ".m2ts", ".mpeg", ".3gp", ".MP4", ".MOV", ".MKV", ".AVI", ".TS", ".FLV", ".WEBM", ".WMV", ".MPG", ".M4V", ".F4V", ".M2TS", ".MPEG", ".3GP"]
OutputFormat = ".mp4" #@param [".mp4", ".mov", ".mkv", ".avi", ".ts", ".flv", ".webm", ".wmv", ".mpg", ".m4v", ".f4v", ".m2ts", ".mpeg", ".3gp", ".MP4", ".MOV", ".MKV", ".AVI", ".TS", ".FLV", ".WEBM", ".WMV", ".MPG", ".M4V", ".F4V", ".M2TS", ".MPEG", ".3GP"]

#@markdown <b><i>Output Options.</i></b>
NumberOfFirstCharactertoRemove = 11 #@param {type:"slider", min:0, max:50}
dchara = NumberOfFirstCharactertoRemove
fix = len(InputFormat)
OverWriteExistingFile = False #@param {type:"boolean"}

#@markdown <b><i>Subtitle Options.</i></b>
BurnIn = True #@param {type:"boolean"}
SubtitleTrack = False #@param {type:"boolean"}
TrackNumber = 0 #@param {type:"slider", min:0, max:10, step:1}

#@markdown <h3><b>Video Quality :</h3>
#@markdown <h><i>Between 0 [Best] to 51 [Worst]</i><h>
Set_cq = '23'#@param {type:"string"}
Set_qmin = '2'#@param {type:"string"}
Set_qmax = '31'#@param {type:"string"}

from pathlib import Path
Path(OutputLocation).mkdir(parents=True, exist_ok=True)

#!ffmpeg -hwaccel cuvid -i '$InputLocation' -c:v h264_nvenc -pix_fmt yuv420p -preset slow -rc vbr_hq -b:v 8M -maxrate:v 10M -c:a aac -b:a 224k '$OutputLocation'$OutputFormat

def OverWrite():
  def noSubtitle():
    cmd = !ffmpeg -y -hwaccel cuvid -i '$InputLocation/{filename}' -c:v h264_nvenc -pix_fmt yuv420p -preset slow -rc vbr_hq -cq $Set_cq -qmin $Set_qmin -qmax $Set_qmax -b:v 8M -maxrate:v 10M -c:a aac -b:a 224k '$OutputLocation/{filename[dchara:-(int(fix))]}'$OutputFormat
  def burnSubtitle():
    def noTrack():
      cmd = !ffmpeg -y -hwaccel cuvid -i '$InputLocation/{filename}' -c:v h264_nvenc -pix_fmt yuv420p -preset slow -rc vbr_hq -cq $Set_cq -qmin $Set_qmin -qmax $Set_qmax -b:v 8M -maxrate:v 10M -c:a aac -b:a 224k -filter_complex "subtitles='$InputLocation/{filename}'" '$OutputLocation/{filename[dchara:-(int(fix))]}'$OutputFormat
    def siTrack():
      cmd = !ffmpeg -y -hwaccel cuvid -i '$InputLocation/{filename}' -c:v h264_nvenc -pix_fmt yuv420p -preset slow -rc vbr_hq -cq $Set_cq -qmin $Set_qmin -qmax $Set_qmax -b:v 8M -maxrate:v 10M -c:a aac -b:a 224k -filter_complex "subtitles='$InputLocation/{filename}:si={TrackNumber}'" '$OutputLocation/{filename[dchara:-(int(fix))]}'$OutputFormat
    if SubtitleTrack == True:
      siTrack()
    if SubtitleTrack == False:
      noTrack()
  if BurnIn == False:
    noSubtitle()
  else:
    burnSubtitle() 
def noOverWrite():
  def noSubtitle():
    cmd = !ffmpeg -n -hwaccel cuvid -i '$InputLocation/{filename}' -c:v h264_nvenc -pix_fmt yuv420p -preset slow -rc vbr_hq -cq $Set_cq -qmin $Set_qmin -qmax $Set_qmax -b:v 8M -maxrate:v 10M -c:a aac -b:a 224k '$OutputLocation/{filename[dchara:-(int(fix))]}'$OutputFormat
  def burnSubtitle():
    def noTrack():
      cmd = !ffmpeg -n -hwaccel cuvid -i '$InputLocation/{filename}' -c:v h264_nvenc -pix_fmt yuv420p -preset slow -rc vbr_hq -cq $Set_cq -qmin $Set_qmin -qmax $Set_qmax -b:v 8M -maxrate:v 10M -c:a aac -b:a 224k -filter_complex "subtitles='$InputLocation/{filename}'" '$OutputLocation/{filename[dchara:-(int(fix))]}'$OutputFormat
    def siTrack():
      cmd = !ffmpeg -n -hwaccel cuvid -i '$InputLocation/{filename}' -c:v h264_nvenc -pix_fmt yuv420p -preset slow -rc vbr_hq -cq $Set_cq -qmin $Set_qmin -qmax $Set_qmax -b:v 8M -maxrate:v 10M -c:a aac -b:a 224k -filter_complex "subtitles='$InputLocation/{filename}:si={TrackNumber}'" '$OutputLocation/{filename[dchara:-(int(fix))]}'$OutputFormat
    if SubtitleTrack == True:
      siTrack()
    else:
      noTrack()  
  if BurnIn == False:
    noSubtitle()
  else:
    burnSubtitle()       
for filename in os.listdir(InputLocation):
  if (filename.endswith(InputFormat)):
    if OverWriteExistingFile == True:
      OverWrite()
    else:
      noOverWrite()

#!ffmpeg -y -hwaccel cuvid -i '$InputLocation' -c:v h264_nvenc -pix_fmt yuv420p -preset slow -rc vbr_hq -cq $Set_cq -qmin $Set_qmin -qmax $Set_qmax -b:v 8M -maxrate:v 10M -c:a aac -b:a 224k -filter_complex "subtitles='$InputLocation'" '$OutputLocation'$OutputFormat

ffmpeg version 3.4.11-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-li

###**» Thumbnail Creation - Preview from Video**

####» Single Video File

In [ ]:
#@markdown <h3><b>Single Video Thumbnail Creation :</h5>
VideoName = "/content/drive/Shareddrives/01. Anime Server/01. Anime/01. TV Show/06. Classroom Of The Elite/Season 2/Streaming Ready/Classroom_Elite_S2_-_02_[1080p_HEVC][Dualsubs].mp4" #@param {type:"string"}
VideoExtension = (".mp4", ".mov", ".mkv", ".avi", ".ts", ".flv", ".webm", ".wmv", ".mpg", ".m4v", ".f4v", ".m2ts", ".mpeg", ".3gp", ".MP4", ".MOV", ".MKV", ".AVI", ".TS", ".FLV", ".WEBM", ".WMV", ".MPG", ".M4V", ".F4V", ".M2TS", ".MPEG", ".3GP")
OutputPath = "/content/drive/Shareddrives/01. Anime Server/01. Anime/01. TV Show/06. Classroom Of The Elite/Season 2/Streaming Ready/Thumbnails" #@param {type:"string"}
ThumbnailName = "Classroom Of The Elite S2 Episode 02" #@param {type:"string"}
PNG = '.png'
JPG = '.jpg'
BMP = '.bmp'
ImageFormat = PNG #@param ["PNG", "JPG", "BMP"] {type:"raw"}
#@markdown <h6><b><i>SingleFrame = Output a single frame from the video into an image file:</i></b></h6>
SingleFrame = False #@param {type:"boolean"}
#@markdown ><h><i>Time Settings:</i><h>
Hours = 0 #@param {type:"slider", min:0, max:10, step:1}
Minutes = 8 #@param {type:"slider", min:0, max:59, step:1}
Seconds = 0 #@param {type:"slider", min:0, max:59, step:1}
Milisec = 0 #@param {type:"slider", min:0, max:999, step:1}


#@markdown <h6><b><i>Output one image every %% :</i></b></h6>
ThumbnailImage = "per 3 minutes" #@param ["per second", "per half a minute", "per minute", "per 3 minutes", "per 10 minutes"]
#@markdown ><h><i>Not used if SingleFrame is checked:</i><h>

from pathlib import Path
Path(OutputPath).mkdir(parents=True, exist_ok=True)

def snglFrame():
  cmd = !ffmpeg -y -hwaccel cuvid -i '{VideoName}' -ss {Hours}:{Minutes}:{Seconds}.{Milisec} -frames:v 1 '$OutputPath/{ThumbnailName}'$ImageFormat
def fpsFrame():
  def scnd():
    cmd = !ffmpeg -y -hwaccel cuvid -i '{VideoName}' -vf fps=1 '$OutputPath/{ThumbnailName}'%03d$ImageFormat
  def hlfmnt():
    cmd = !ffmpeg -y -hwaccel cuvid -i '{VideoName}' -vf fps=2/60 '$OutputPath/{ThumbnailName}'%03d$ImageFormat
  def mnt():
    cmd = !ffmpeg -y -hwaccel cuvid -i '{VideoName}' -vf fps=1/60 '$OutputPath/{ThumbnailName}'%03d$ImageFormat
  def thrmnt():
    cmd = !ffmpeg -y -hwaccel cuvid -i '{VideoName}' -vf fps=1/180 '$OutputPath/{ThumbnailName}'%03d$ImageFormat
  def tenmnt():
    cmd = !ffmpeg -y -hwaccel cuvid -i '{VideoName}' -vf fps=1/600 '$OutputPath/{ThumbnailName}'%03d$ImageFormat
  if ThumbnailImage == "per second":
    scnd()
  if ThumbnailImage == "per half a minute":
    hlfmnt()
  if ThumbnailImage == "per minute":
    mnt()
  if ThumbnailImage == "per 3 minutes":
    thrmnt()
  if ThumbnailImage == "per 10 minutes":
    tenmnt()
if SingleFrame == True:
  snglFrame()
else:
  fpsFrame()



####» Entire videos in the directory

In [ ]:
#@markdown <h3><b>Full Directory Thumbnail Creation :</h5>
InputPath = "/content/drive/Shareddrives/01. Anime Server/01. TV Show/38. Fuuto Tantei/Season 1" #@param {type:"string"}
VideoExtension = ".mkv" #@param [".mp4", ".mov", ".mkv", ".avi", ".ts", ".flv", ".webm", ".wmv", ".mpg", ".m4v", ".f4v", ".m2ts", ".mpeg", ".3gp", ".MP4", ".MOV", ".MKV", ".AVI", ".TS", ".FLV", ".WEBM", ".WMV", ".MPG", ".M4V", ".F4V", ".M2TS", ".MPEG", ".3GP"]
OutputPath = "/content/drive/Shareddrives/01. Anime Server/01. TV Show/38. Fuuto Tantei/Season 1/Thumbnails" #@param {type:"string"}
OverWriteExistingFile = False #@param {type:"boolean"}
NumberOfFirstCharactertoRemove = 11 #@param {type:"slider", min:0, max:50}
PNG = '.png'
JPG = '.jpg'
BMP = '.bmp'
ImageFormat = JPG #@param ["PNG", "JPG", "BMP"] {type:"raw"}
#@markdown <h6><b><i>SingleFrame = Output a single frame from the video into an image file:</i></b></h6>
SingleFrame = True #@param {type:"boolean"}
#@markdown ><h><i>Time Settings:</i><h>
Hours = 0 #@param {type:"slider", min:0, max:10, step:1}
Minutes = 9 #@param {type:"slider", min:0, max:59, step:1}
Seconds = 10 #@param {type:"slider", min:0, max:59, step:1}
Milisec = 185 #@param {type:"slider", min:0, max:999, step:1}


#@markdown <h6><b><i>Output one image every %% :</i></b></h6>
ThumbnailImage = "per minute" #@param ["per second", "per half a minute", "per minute", "per 3 minutes", "per 10 minutes"]
#@markdown ><h><i>Not used if SingleFrame is checked:</i><h>

dchara = NumberOfFirstCharactertoRemove
fix = len(InputFormat)

from pathlib import Path
Path(OutputPath).mkdir(parents=True, exist_ok=True)

def overWriting():
  def snglFrame():
    cmd = !ffmpeg -y -hwaccel cuvid -i '$InputPath/{filename}' -ss {Hours}:{Minutes}:{Seconds}.{Milisec} -frames:v 1 '$OutputPath/{filename[dchara:-(int(fix))]}'$ImageFormat
  def fpsFrame():
    def scnd():
      cmd = !ffmpeg -y -hwaccel cuvid -i '$InputPath/{filename}' -vf fps=1 '$OutputPath/{filename[dchara:-(int(fix))]}'%03d$ImageFormat
    def hlfmnt():
      cmd = !ffmpeg -y -hwaccel cuvid -i '$InputPath/{filename}' -vf fps=2/60 '$OutputPath/{filename[dchara:-(int(fix))]}'%03d$ImageFormat
    def mnt():
      cmd = !ffmpeg -y -hwaccel cuvid -i '$InputPath/{filename}' -vf fps=1/60 '$OutputPath/{filename[dchara:-(int(fix))]}'%03d$ImageFormat
    def thrmnt():
      cmd = !ffmpeg -y -hwaccel cuvid -i '$InputPath/{filename}' -vf fps=1/180 '$OutputPath/{filename[dchara:-(int(fix))]}'%03d$ImageFormat
    def tenmnt():
      cmd = !ffmpeg -y -hwaccel cuvid -i '$InputPath/{filename}' -vf fps=1/600 '$OutputPath/{filename[dchara:-(int(fix))]}'%03d$ImageFormat
    if ThumbnailImage == "per second":
      scnd()
    if ThumbnailImage == "per half a minute":
      hlfmnt()
    if ThumbnailImage == "per minute":
      mnt()
    if ThumbnailImage == "per 3 minutes":
      thrmnt()
    if ThumbnailImage == "per 10 minutes":
      tenmnt()
  if SingleFrame == True:
    snglFrame()
  else:
    fpsFrame()
def NooverWriting():
  def snglFrame():
    cmd = !ffmpeg -n -hwaccel cuvid -i '$InputPath/{filename}' -ss {Hours}:{Minutes}:{Seconds}.{Milisec} -frames:v 1 '$OutputPath/{filename[dchara:-(int(fix))]}'$ImageFormat
  def fpsFrame():
    def scnd():
      cmd = !ffmpeg -n -hwaccel cuvid -i '$InputPath/{filename}' -vf fps=1 '$OutputPath/{filename[dchara:-(int(fix))]}'%03d$ImageFormat
    def hlfmnt():
      cmd = !ffmpeg -n -hwaccel cuvid -i '$InputPath/{filename}' -vf fps=2/60 '$OutputPath/{filename[dchara:-(int(fix))]}'%03d$ImageFormat
    def mnt():
      cmd = !ffmpeg -n -hwaccel cuvid -i '$InputPath/{filename}' -vf fps=1/60 '$OutputPath/{filename[dchara:-(int(fix))]}'%03d$ImageFormat
    def thrmnt():
      cmd = !ffmpeg -n -hwaccel cuvid -i '$InputPath/{filename}' -vf fps=1/180 '$OutputPath/{filename[dchara:-(int(fix))]}'%03d$ImageFormat
    def tenmnt():
      cmd = !ffmpeg -n -hwaccel cuvid -i '$InputPath/{filename}' -vf fps=1/600 '$OutputPath/{filename[dchara:-(int(fix))]}'%03d$ImageFormat
    if ThumbnailImage == "per second":
      scnd()
    if ThumbnailImage == "per half a minute":
      hlfmnt()
    if ThumbnailImage == "per minute":
      mnt()
    if ThumbnailImage == "per 3 minutes":
      thrmnt()
    if ThumbnailImage == "per 10 minutes":
      tenmnt()
  if SingleFrame == True:
    snglFrame()
  else:
    fpsFrame()
for filename in os.listdir(InputPath):
  if (filename.endswith(VideoExtension)):
    if OverWriteExistingFile == True:
      overWriting()
    else:
      NooverWriting()

###» Extract Subtitle from Video 




In [ ]:
#@markdown <h3>← Verify Tracks for Video</h3>
import os, sys, re
video_file_path = "/content/drive/Shareddrives/01. Anime Server/04. Anime China/War God System!/War God System! - 01 (B-Global Donghua 1920x1080 HEVC AAC MKV).mkv" #@param {type:"string"}

!mkvmerge -i "$video_file_path"

In [ ]:
import os, sys, re

video_file_path = "/content/drive/Shareddrives/01. Anime Server/04. Anime China/War God System!/War God System! - 01 (B-Global Donghua 1920x1080 HEVC AAC MKV).mkv" #@param {type:"string"}
output_file_type = 'ass' #@param ["srt", "ass", "idx/sub"]

output_file_path = re.search("^[\/].+\/", video_file_path)
output_file_path_raw = output_file_path.group(0)
delsplit = re.search("\/(?:.(?!\/))+$", video_file_path)
filename = re.sub("^[\/]", "", delsplit.group(0))
filename_raw = re.sub(".{4}$", "", filename)
file_extension = re.search(".{3}$", filename)
file_extension_raw = file_extension.group(0)

os.environ['inputFile'] = video_file_path
os.environ['outputPath'] = output_file_path_raw
os.environ['outputExtension'] = output_file_type
os.environ['fileName'] = filename_raw
os.environ['fileExtension'] = file_extension_raw

if output_file_type == 'srt':
    !ffmpeg -hide_banner -i "$inputFile" "$outputPath"/"$fileName"."$outputExtension"

if output_file_type == 'ass':
    !ffmpeg -hide_banner -i "$inputFile" "$outputPath"/"$fileName"."$outputExtension"

if output_file_type == 'idx/sub':
    !mkvextract "$inputFile" tracks 2:"$outputPath"/"$fileName".idx


###» Video to (*H2.64*-AAC)  - Need GPU - Nvidia Telsa P100 or T4

In [ ]:
#@markdown <h3><b>Full Folder Encode :</h3>
#@markdown <i>Put file location and name (include the extension). Convert in the entire directory.</i>

import os, sys, re

input_folder= '/content/drive/Shareddrives/01. Anime Server/01. Anime/01. TV Show/36. DanMachi/Season 4' #@param {type:"string"}
output_folder= '/content/VAAPI' #@param {type:"string"}
#@markdown <h3><b>Hardware Accelerator Setting :</h3>
#hwaccel setting
VDPAU = 'vdpau'
VAAPI = 'vaapi'
CUVID = 'cuvid'
HardwareAccelerator = VAAPI #@param ["VDPAU", "VAAPI", "CUVID"] {type:"raw"}
#@markdown <h3><b>Choose your convert setting :</h5>
#@markdown <b><i>Output options</i></b>
Overwrite_Option = "Don't Overwrite an Existing File" #@param ["Default", "Overwrite an Existing File", "Don't Overwrite an Existing File"]
Convert_Method = "With Subtitle" #@param ["No Subtitle", "With Subtitle"]
#@markdown <b><i>Subtitle options</i></b>
NumberOfFirstCharactertoRemove = 0 #@param {type:"slider", min:0, max:50}
HideSubtitleTrack = True #@param {type:"boolean"}
SubtitleTrack = 0 #@param {type:"slider", min:0, max:10}
#@markdown <h3><b>Choose your encode setting :</h5>
#@markdown <h><i>main = default | main10 = H264 10 Bit</i>
H264 = 'h264_nvenc'
HEVC = 'hevc_nvenc'
Normal = ''
encode_setting = H264 #@param ["H264", "HEVC", "Normal"] {type:"raw"}
set_profile = "main" #@param ["main", "main10"]
input_file_type = ".mkv" #@param [".mp4", ".mov", ".mkv", ".avi", ".ts", ".flv", ".webm", ".wmv", ".mpg", ".m4v", ".f4v", ".m2ts", ".mpeg", ".3gp", ".MP4", ".MOV", ".MKV", ".AVI", ".TS", ".FLV", ".WEBM", ".WMV", ".MPG", ".M4V", ".F4V", ".M2TS", ".MPEG", ".3GP"]
my_suffixes = input_file_type
output_file_type = ".mp4" #@param [".mp4", ".mov", ".mkv", ".avi", ".ts", ".flv", ".webm", ".wmv", ".mpg", ".m4v", ".f4v", ".m2ts", ".mpeg", ".3gp", ".MP4", ".MOV", ".MKV", ".AVI", ".TS", ".FLV", ".WEBM", ".WMV", ".MPG", ".M4V", ".F4V", ".M2TS", ".MPEG", ".3GP"]
#@markdown <h3><b>Video quality :</h3>
#@markdown <h><i>0 (Best) to 51 (Worst)</i><h>
set_cq = '22'#@param {type:"string"}
set_qmin = '22'#@param {type:"string"}
set_qmax = '24'#@param {type:"string"}

#@markdown <h3><b>Encoding Process :</h3>
#@markdown <h><i>Output screen information</i><h>
Show = ''
Hide = '-v error'
Status = Hide #@param ["Show", "Hide"] {type:"raw"}

dchara = NumberOfFirstCharactertoRemove
fix = len(input_file_type)
    
from pathlib import Path
Path(output_folder).mkdir(parents=True, exist_ok=True)

def noSub():
  def owdef():
    cmd = !ffmpeg {Status} -stats -hwaccel {HardwareAccelerator} -i "$input_folder/{filename}" -c:v {encode_setting} -preset:v slow -rc vbr -cq $set_cq -qmin $set_qmin -qmax $set_qmax -b:v 0 -profile:v $set_profile -init_qpP 23 -init_qpB 25 -init_qpI 21 -c:a copy "$output_folder/{filename[dchara:-(int(fix))]}{output_file_type}"
  def owyes():   
    cmd = !ffmpeg {Status} -stats -hwaccel {HardwareAccelerator} -i "$input_folder/{filename}" -c:v {encode_setting} -preset:v slow -rc vbr -cq $set_cq -qmin $set_qmin -qmax $set_qmax -b:v 0 -profile:v $set_profile -init_qpP 23 -init_qpB 25 -init_qpI 21 -c:a copy "$output_folder/{filename[dchara:-(int(fix))]}{output_file_type}" -y
  def owno():
    cmd = !ffmpeg {Status} -stats -hwaccel {HardwareAccelerator} -i "$input_folder/{filename}" -c:v {encode_setting} -preset:v slow -rc vbr -cq $set_cq -qmin $set_qmin -qmax $set_qmax -b:v 0 -profile:v $set_profile -init_qpP 23 -init_qpB 25 -init_qpI 21 -c:a copy "$output_folder/{filename[dchara:-(int(fix))]}{output_file_type}" -n
  if Overwrite_Option == "Default":
    owdef()
  if Overwrite_Option == "Overwrite an Existing File":
    owyes()
  if Overwrite_Option == "Don't Overwrite an Existing File":
    owno()

def withSub():
  def noSubTrack():
    def owdef():
      cmd = !ffmpeg {Status} -stats -hwaccel {HardwareAccelerator} -i "$input_folder/{filename}" -c:v {encode_setting} -preset:v slow -rc vbr -cq $set_cq -qmin $set_qmin -qmax $set_qmax -b:v 0 -profile:v $set_profile -init_qpP 23 -init_qpB 25 -init_qpI 21 -c:a copy -filter_complex "subtitles='$input_folder/{filename}'" "$output_folder/{filename[dchara:-(int(fix))]}{output_file_type}"
    def owyes():
      cmd = !ffmpeg {Status} -stats -hwaccel {HardwareAccelerator} -i "$input_folder/{filename}" -c:v {encode_setting} -preset:v slow -rc vbr -cq $set_cq -qmin $set_qmin -qmax $set_qmax -b:v 0 -profile:v $set_profile -init_qpP 23 -init_qpB 25 -init_qpI 21 -c:a copy -filter_complex "subtitles='$input_folder/{filename}'" "$output_folder/{filename[dchara:-(int(fix))]}{output_file_type}" -y
    def owno():
      cmd = !ffmpeg {Status} -stats -hwaccel {HardwareAccelerator} -i "$input_folder/{filename}" -c:v {encode_setting} -preset:v slow -rc vbr -cq $set_cq -qmin $set_qmin -qmax $set_qmax -b:v 0 -profile:v $set_profile -init_qpP 23 -init_qpB 25 -init_qpI 21 -c:a copy -filter_complex "subtitles='$input_folder/{filename}'" "$output_folder/{filename[dchara:-(int(fix))]}{output_file_type}" -n
    if Overwrite_Option == "Default":
      owdef()
    if Overwrite_Option == "Overwrite an Existing File":
      owyes()
    if Overwrite_Option == "Don't Overwrite an Existing File":
      owno()
  def SubTrack():
    def owdef():
      cmd = !ffmpeg {Status} -stats -hwaccel {HardwareAccelerator} -i "$input_folder/{filename}" -c:v {encode_setting} -preset:v slow -rc vbr -cq $set_cq -qmin $set_qmin -qmax $set_qmax -b:v 0 -profile:v $set_profile -init_qpP 23 -init_qpB 25 -init_qpI 21 -c:a copy -filter_complex "subtitles='$input_folder/{filename}:si={SubtitleTrack}'" "$output_folder/{filename[dchara:-(int(fix))]}{output_file_type}"
    def owyes():
      cmd = !ffmpeg {Status} -stats -hwaccel {HardwareAccelerator} -i "$input_folder/{filename}" -c:v {encode_setting} -preset:v slow -rc vbr -cq $set_cq -qmin $set_qmin -qmax $set_qmax -b:v 0 -profile:v $set_profile -init_qpP 23 -init_qpB 25 -init_qpI 21 -c:a copy -filter_complex "subtitles='$input_folder/{filename}:si={SubtitleTrack}'" "$output_folder/{filename[dchara:-(int(fix))]}{output_file_type}" -y
    def owno():
      cmd = !ffmpeg {Status} -stats -hwaccel {HardwareAccelerator} -i "$input_folder/{filename}" -c:v {encode_setting} -preset:v slow -rc vbr -cq $set_cq -qmin $set_qmin -qmax $set_qmax -b:v 0 -profile:v $set_profile -init_qpP 23 -init_qpB 25 -init_qpI 21 -c:a copy -filter_complex "subtitles='$input_folder/{filename}:si={SubtitleTrack}'" "$output_folder/{filename[dchara:-(int(fix))]}{output_file_type}" -n
    if Overwrite_Option == "Default":
      owdef()
    if Overwrite_Option == "Overwrite an Existing File":
      owyes()
    if Overwrite_Option == "Don't Overwrite an Existing File":
      owno()
  if HideSubtitleTrack == True:
    noSubTrack()
  if HideSubtitleTrack == False:
    SubTrack()

for filename in os.listdir(input_folder):
  if (filename.endswith(my_suffixes)):
        if Convert_Method == "No Subtitle":
          noSub()
        if Convert_Method == "With Subtitle":
          withSub()


###» Video to (*HEVC*-AAC)  - Need GPU - Nvidia Telsa P100 or T4

In [ ]:
#@markdown <h3><b>Full Folder Encode :</h3>

import os, sys, re

input_folder= '/content/drive/Shareddrives/01. Anime Server/01. Anime/01. TV Show/23. Yofukashi no Uta/Season1' #@param {type:"string"}
output_folder= '/content/drive/Shareddrives/01. Anime Server/01. Anime/01. TV Show/23. Yofukashi no Uta/Season1/TEST' #@param {type:"string"}
#@markdown <h3><b>Choose your encode setting :</h5>
#@markdown <h><i>main = default | main10 = HEVC 10 Bit</i>
encode_setting = "HEVC (Audio Copy)" #@param ["HEVC AAC 160k", "HEVC AAC_adtstoasc", "HEVC (Audio Copy)", "AAC 160k (Video Copy)"]
set_profile = "main" #@param ["main", "main10"]
input_file_type = ".mp4" #@param [".mp4", ".mov", ".mkv", ".avi", ".ts", ".flv", ".webm", ".wmv", ".mpg", ".m4v", ".f4v", ".m2ts", ".mpeg", ".3gp", ".MP4", ".MOV", ".MKV", ".AVI", ".TS", ".FLV", ".WEBM", ".WMV", ".MPG", ".M4V", ".F4V", ".M2TS", ".MPEG", ".3GP"]
my_suffixes = input_file_type
output_file_type = ".mkv" #@param [".mp4", ".mov", ".mkv", ".avi", ".ts", ".flv", ".webm", ".wmv", ".mpg", ".m4v", ".f4v", ".m2ts", ".mpeg", ".3gp", ".MP4", ".MOV", ".MKV", ".AVI", ".TS", ".FLV", ".WEBM", ".WMV", ".MPG", ".M4V", ".F4V", ".M2TS", ".MPEG", ".3GP"]
#@markdown <h3><b>Video quality :</h3>
#@markdown <h><i>0 (Best) to 51 (Worst)</i><h>
set_cq = '18'#@param {type:"string"}
set_qmin = '18'#@param {type:"string"}
set_qmax = '20'#@param {type:"string"}

from pathlib import Path
Path(output_folder).mkdir(parents=True, exist_ok=True)

if encode_setting == 'HEVC AAC 160k':
 for filename in os.listdir(input_folder):
    if (filename.endswith(my_suffixes)):
        cmd = !ffmpeg -v error -stats -hwaccel cuvid -i "$input_folder/{filename}" -metadata comment="HEVC-CQ$set_cq-AAC" -c:v hevc_nvenc -preset:v slow -rc vbr -cq $set_cq -qmin $set_qmin -qmax $set_qmax -b:v 0 -profile:v $set_profile -init_qpP 23 -init_qpB 25 -init_qpI 21 -c:a aac -b:a 160k "$output_folder/{filename.rpartition('.')[0]}.$output_file_type"

if encode_setting == 'HEVC AAC_adtstoasc':
 for filename in os.listdir(input_folder):
    if (filename.endswith(my_suffixes)):
        cmd = !ffmpeg -v error -stats -hwaccel cuvid -i "$input_folder/{filename}" -metadata comment="HEVC-CQ$set_cq-AAC" -c:v hevc_nvenc -preset:v slow -rc vbr -cq $set_cq -qmin $set_qmin -qmax $set_qmax -b:v 0 -profile:v $set_profile -init_qpP 23 -init_qpB 25 -init_qpI 21 -bsf:a aac_adtstoasc "$output_folder/{filename.rpartition('.')[0]}.$output_file_type"

if encode_setting == 'HEVC (Audio Copy)':
 for filename in os.listdir(input_folder):
    if (filename.endswith(my_suffixes)):
        cmd = !ffmpeg -v error -stats -hwaccel cuvid -i "$input_folder/{filename}" -metadata comment="HEVC-CQ$set_cq-AAC" -c:v hevc_nvenc -preset:v slow -rc vbr -cq $set_cq -qmin $set_qmin -qmax $set_qmax -b:v 0 -profile:v $set_profile -init_qpP 23 -init_qpB 25 -init_qpI 21 -c:a copy "$output_folder/{filename.rpartition('.')[0]}.$output_file_type"

if encode_setting == 'AAC 160k (Video Copy)':
 for filename in os.listdir(input_folder):
    if (filename.endswith(my_suffixes)):
        cmd = !ffmpeg -v error -stats -hwaccel cuvid -i "$input_folder/{filename}" -metadata comment="HEVC-CQ$set_cq-AAC" -c:v copy -c:a aac -b:a 160k "$output_folder/{filename.rpartition('.')[0]}.$output_file_type"
                     


###» Video to (*X265*-AAC)  - No GPU

In [ ]:
#@markdown <h3><b>Full Folder Encode :</h3>

import os, sys, re

input_folder= '/content/drive/Shareddrives/01. Anime Server/01. Anime/01. TV Show/23. Yofukashi no Uta/Season1/TEST' #@param {type:"string"}
output_folder= '/content/drive/Shareddrives/01. Anime Server/01. Anime/01. TV Show/23. Yofukashi no Uta/Season1/TEST/ok' #@param {type:"string"}
#@markdown <h3><b>Choose your encode setting :</h5>
#@markdown <h><i>main = default | main10 = HEVC 10 Bit</i>
encode_setting = "X265 (Audio Copy)" #@param ["X265 AAC 160k", "X265 AAC_adtstoasc", "X265 (Audio Copy)", "AAC 160k (Video Copy)"]
output_file_type = 'mp4' #@param ["mkv", "mp4"]
#@markdown <h3><b>Video quality :</h3>
#@markdown <h><i>0 (Best) to 51 (Worst)</i><h>
set_crf ='26'#@param {type:"string"}
my_suffixes = (".mp4", ".mov", ".mkv", ".avi", ".ts", ".flv", ".webm", ".wmv", ".mpg", ".m4v", ".f4v", ".m2ts", ".mpeg", ".3gp", ".MP4", ".MOV", ".MKV", ".AVI", ".TS", ".FLV", ".WEBM", ".WMV", ".MPG", ".M4V", ".F4V", ".M2TS", ".MPEG", ".3GP")

from pathlib import Path
Path(output_folder).mkdir(parents=True, exist_ok=True)

if encode_setting == 'X265 AAC 160k':
 for filename in os.listdir(input_folder):
    if (filename.endswith(my_suffixes)):
        cmd = !ffmpeg -v error -stats -i "$input_folder/{filename}" -metadata comment="X265-CRF$set_crf-AAC" -c:v libx265 -preset:v slow -crf $set_crf -c:a aac -b:a 160k "$output_folder/{filename.rpartition('.')[0]}.$output_file_type"

if encode_setting == 'X265 AAC_adtstoasc':
 for filename in os.listdir(input_folder):
    if (filename.endswith(my_suffixes)):
        cmd = !ffmpeg -v error -stats -i "$input_folder/{filename}" -metadata comment="X265-CRF$set_crf-AAC" -c:v libx265 -preset:v slow -crf $set_crf -bsf:a aac_adtstoasc "$output_folder/{filename.rpartition('.')[0]}.$output_file_type"

if encode_setting == 'X265 (Audio Copy)':
 for filename in os.listdir(input_folder):
    if (filename.endswith(my_suffixes)):
        cmd = !ffmpeg -v error -stats -i "$input_folder/{filename}" -metadata comment="X265-CRF$set_crf-AAC" -c:v libx265 -preset:v slow -crf $set_crf -c:a copy "$output_folder/{filename.rpartition('.')[0]}.$output_file_type"

if encode_setting == 'AAC 160k (Video Copy)':
 for filename in os.listdir(input_folder):
    if (filename.endswith(my_suffixes)):
        cmd = !ffmpeg -v error -stats -i "$input_folder/{filename}" -metadata comment="VideoCopy-AAC" -c:v copy -c:a aac -b:a 160k "$output_folder/{filename.rpartition('.')[0]}.$output_file_type"
                     

###» Crop Video to (*H2.64*-AAC) - Need GPU - Nvidia Telsa P100 or T4

In [ ]:
#@markdown <h3><b>Full Folder Encode with Crop :</h3>
#@markdown <i>Put file location and name (include the extension). Convert in the entire directory.</i>

import os, sys, re

input_folder= '/content/drive/Shareddrives/01. Anime Server/01. Anime/01. TV Show/23. Yofukashi no Uta/Season1' #@param {type:"string"}
output_folder= '/content/drive/Shareddrives/01. Anime Server/01. Anime/01. TV Show/23. Yofukashi no Uta/Season1/TEST' #@param {type:"string"}
#@markdown <h3><b>Choose your convert setting :</h5>
#@markdown <h><i>main = default | main10 = H264 10 Bit</i>
Overwrite_Option = "Overwrite an Existing File" #@param ["Default", "Overwrite an Existing File", "Don't Overwrite an Existing File"]
Convert_Method = "With Subtitle" #@param ["No Subtitle", "With Subtitle"]
SubtitleTrack = 0 #@param {type:"slider", min:0, max:10}
NumberOfFirstCharactertoRemove = 0 #@param {type:"slider", min:0, max:50}
#@markdown <h3><b>Choose your encode setting :</h5>
#@markdown <h><i>main = default | main10 = H264 10 Bit</i>
set_profile = "main" #@param ["main", "main10"]
output_file_type = 'mp4' #@param ["mkv", "mp4"]
#@markdown <h3><b>Video quality :</h3>
#@markdown <h><i>0 (Best) to 51 (Worst)</i><h>
set_cq = '28'#@param {type:"string"}
set_qmin = '28'#@param {type:"string"}
set_qmax = '30'#@param {type:"string"}
my_suffixes = (".mp4", ".mov", ".mkv", ".avi", ".ts", ".flv", ".webm", ".wmv", ".mpg", ".m4v", ".f4v", ".m2ts", ".mpeg", ".3gp", ".MP4", ".MOV", ".MKV", ".AVI", ".TS", ".FLV", ".WEBM", ".WMV", ".MPG", ".M4V", ".F4V", ".M2TS", ".MPEG", ".3GP")
#@markdown <h3><b>Crop Settings :</h3>
crop_w = '0'#@param {type:"string"}
crop_h = '0'#@param {type:"string"}
crop_x = '0'#@param {type:"string"}
crop_y = '0'#@param {type:"string"}

dchara = NumberOfFirstCharactertoRemove
fix = len(input_file_type)

from pathlib import Path
Path(output_folder).mkdir(parents=True, exist_ok=True)

def noSub():
  def owdef():
    cmd = !ffmpeg -v error -stats -hwaccel cuvid -i "$input_folder/{filename}" -metadata comment="HEVC-CQ$set_cq-AAC" -c:v h264_nvenc -preset:v slow -rc vbr -cq $set_cq -qmin $set_qmin -qmax $set_qmax -b:v 0 -profile:v $set_profile -init_qpP 23 -init_qpB 25 -init_qpI 21 -filter:v "crop=$crop_w:$crop_h:$crop_x:$crop_y" -c:a copy "$output_folder/{filename[dchara:-(int(fix))]}{output_file_type}"
  def owyes():
    cmd = !ffmpeg -v error -stats -hwaccel cuvid -i "$input_folder/{filename}" -metadata comment="HEVC-CQ$set_cq-AAC" -c:v h264_nvenc -preset:v slow -rc vbr -cq $set_cq -qmin $set_qmin -qmax $set_qmax -b:v 0 -profile:v $set_profile -init_qpP 23 -init_qpB 25 -init_qpI 21 -filter:v "crop=$crop_w:$crop_h:$crop_x:$crop_y" -c:a copy "$output_folder/{filename[dchara:-(int(fix))]}{output_file_type}" -y
  def owno():
    cmd = !ffmpeg -v error -stats -hwaccel cuvid -i "$input_folder/{filename}" -metadata comment="HEVC-CQ$set_cq-AAC" -c:v h264_nvenc -preset:v slow -rc vbr -cq $set_cq -qmin $set_qmin -qmax $set_qmax -b:v 0 -profile:v $set_profile -init_qpP 23 -init_qpB 25 -init_qpI 21 -filter:v "crop=$crop_w:$crop_h:$crop_x:$crop_y" -c:a copy "$output_folder/{filename[dchara:-(int(fix))]}{output_file_type}" -n
  if Overwrite_Option == "Default":
    owdef()
  if Overwrite_Option == "Overwrite an Existing File":
    owyes()
  if Overwrite_Option == "Don't Overwrite an Existing File":
    owno()

def withSub():
  def owdef():
    cmd = !ffmpeg -v error -stats -hwaccel cuvid -i "$input_folder/{filename}" -metadata comment="HEVC-CQ$set_cq-AAC" -c:v h264_nvenc -preset:v slow -rc vbr -cq $set_cq -qmin $set_qmin -qmax $set_qmax -b:v 0 -profile:v $set_profile -init_qpP 23 -init_qpB 25 -init_qpI 21 -filter:v "crop=$crop_w:$crop_h:$crop_x:$crop_y" -c:a copy -filter_complex "subtitles='$input_folder/{filename}:si={SubtitleTrack}'" "$output_folder/{filename[dchara:-(int(fix))]}{output_file_type}"
  def owyes():
    cmd = !ffmpeg -v error -stats -hwaccel cuvid -i "$input_folder/{filename}" -metadata comment="HEVC-CQ$set_cq-AAC" -c:v h264_nvenc -preset:v slow -rc vbr -cq $set_cq -qmin $set_qmin -qmax $set_qmax -b:v 0 -profile:v $set_profile -init_qpP 23 -init_qpB 25 -init_qpI 21 -filter:v "crop=$crop_w:$crop_h:$crop_x:$crop_y" -c:a copy -filter_complex "subtitles='$input_folder/{filename}:si={SubtitleTrack}'" "$output_folder/{filename[dchara:-(int(fix))]}{output_file_type}" -y
  def owno():
    cmd = !ffmpeg -v error -stats -hwaccel cuvid -i "$input_folder/{filename}" -metadata comment="HEVC-CQ$set_cq-AAC" -c:v h264_nvenc -preset:v slow -rc vbr -cq $set_cq -qmin $set_qmin -qmax $set_qmax -b:v 0 -profile:v $set_profile -init_qpP 23 -init_qpB 25 -init_qpI 21 -filter:v "crop=$crop_w:$crop_h:$crop_x:$crop_y" -c:a copy -filter_complex "subtitles='$input_folder/{filename}:si={SubtitleTrack}'" "$output_folder/{filename[dchara:-(int(fix))]}{output_file_type}" -n
  if Overwrite_Option == "Default":
    owdef()
  if Overwrite_Option == "Overwrite an Existing File":
    owyes()
  if Overwrite_Option == "Don't Overwrite an Existing File":
    owno()

for filename in os.listdir(input_folder):
  if (filename.endswith(my_suffixes)):
        if Convert_Method == "No Subtitle":
          noSub()
        if Convert_Method == "With Subtitle":
          withSub()

###» Crop Video to (*HEVC*-AAC) - Need GPU - Nvidia Telsa P100 or T4

In [ ]:
#@markdown <h3><b>Full Folder Encode with Crop :</h3>

import os, sys, re

input_folder= '/content/drive/My Drive/Videos/' #@param {type:"string"}
output_folder= '/content/drive/My Drive/Videos/HEVC' #@param {type:"string"}
#@markdown <h3><b>Choose your encode setting :</h5>
#@markdown <h><i>main = default | main10 = HEVC 10 Bit</i>
encode_setting = "HEVC (Audio Copy)" #@param ["HEVC AAC 160k", "HEVC AAC_adtstoasc", "HEVC (Audio Copy)"]
set_profile = "main" #@param ["main", "main10"]
output_file_type = 'mp4' #@param ["mkv", "mp4"]
#@markdown <h3><b>Video quality :</h3>
#@markdown <h><i>0 (Best) to 51 (Worst)</i><h>
set_cq = '28'#@param {type:"string"}
set_qmin = '28'#@param {type:"string"}
set_qmax = '30'#@param {type:"string"}
my_suffixes = (".mp4", ".mov", ".mkv", ".avi", ".ts", ".flv", ".webm", ".wmv", ".mpg", ".m4v", ".f4v", ".m2ts", ".mpeg", ".3gp", ".MP4", ".MOV", ".MKV", ".AVI", ".TS", ".FLV", ".WEBM", ".WMV", ".MPG", ".M4V", ".F4V", ".M2TS", ".MPEG", ".3GP")
#@markdown <h3><b>Crop Settings :</h3>
crop_w = '0'#@param {type:"string"}
crop_h = '0'#@param {type:"string"}
crop_x = '0'#@param {type:"string"}
crop_y = '0'#@param {type:"string"}

from pathlib import Path
Path(output_folder).mkdir(parents=True, exist_ok=True)

if encode_setting == 'HEVC AAC 160k':
 for filename in os.listdir(input_folder):
    if (filename.endswith(my_suffixes)):
        cmd = !ffmpeg -v quiet -stats -hwaccel cuvid -i "$input_folder/{filename}" -metadata comment="HEVC-CQ$set_cq-AAC" -c:v hevc_nvenc -preset:v slow -rc vbr -cq $set_cq -qmin $set_qmin -qmax $set_qmax -b:v 0 -profile:v $set_profile -init_qpP 23 -init_qpB 25 -init_qpI 21 -filter:v "crop=$crop_w:$crop_h:$crop_x:$crop_y" -c:a aac -b:a 160k "$output_folder/{filename.rpartition('.')[0]}.$output_file_type"

if encode_setting == 'HEVC AAC_adtstoasc':
 for filename in os.listdir(input_folder):
    if (filename.endswith(my_suffixes)):
        cmd = !ffmpeg -v quiet -stats -hwaccel cuvid -i "$input_folder/{filename}" -metadata comment="HEVC-CQ$set_cq-AAC" -c:v hevc_nvenc -preset:v slow -rc vbr -cq $set_cq -qmin $set_qmin -qmax $set_qmax -b:v 0 -profile:v $set_profile -init_qpP 23 -init_qpB 25 -init_qpI 21 -filter:v "crop=$crop_w:$crop_h:$crop_x:$crop_y" -bsf:a aac_adtstoasc "$output_folder/{filename.rpartition('.')[0]}.$output_file_type"

if encode_setting == 'HEVC (Audio Copy)':
 for filename in os.listdir(input_folder):
    if (filename.endswith(my_suffixes)):
        cmd = !ffmpeg -v quiet -stats -hwaccel cuvid -i "$input_folder/{filename}" -metadata comment="HEVC-CQ$set_cq-AAC" -c:v hevc_nvenc -preset:v slow -rc vbr -cq $set_cq -qmin $set_qmin -qmax $set_qmax -b:v 0 -profile:v $set_profile -init_qpP 23 -init_qpB 25 -init_qpI 21 -filter:v "crop=$crop_w:$crop_h:$crop_x:$crop_y" -c:a copy "$output_folder/{filename.rpartition('.')[0]}.$output_file_type"
                     

###» Display Media File Codecs + Metadata

In [ ]:
import os, sys, re

media_file_path = "" #@param {type:"string"}

os.environ['inputFile'] = media_file_path

!ffmpeg -i "$inputFile" -hide_banner

> *You can ignore the "At least one output file must be specified" error after running this.*


###» Generate Thumbnails - Preview from Video 

In [ ]:
#@markdown <h3>← Click Here to generate thumbnail for selected video</h3>


import os, sys, re
from IPython.display import Image, display
os.makedirs("/content/drive/My Drive/Thumbnail", exist_ok=True)

video_file_path = "" #@param {type:"string"}
output_file_type = 'png' #@param ["png", "jpg"]
creation_engine = 'vcsi' #@param ["ffmpeg", "vcsi"]
output_path = 'same folder' #@param ["same folder", "My Drive/Thumbnail"]

output_file_path = re.search("^[\/].+\/", video_file_path)
output_file_path_raw = output_file_path.group(0)
delsplit = re.search("\/(?:.(?!\/))+$", video_file_path)
filename = re.sub("^[\/]", "", delsplit.group(0))
filename_raw = re.sub(".{4}$", "", filename)
file_extension = re.search(".{3}$", filename)
file_extension_raw = file_extension.group(0)

os.environ['inputFile'] = video_file_path
os.environ['outputPath'] = output_file_path_raw
os.environ['outputExtension'] = output_file_type
os.environ['fileName'] = filename_raw
os.environ['fileExtension'] = file_extension_raw

if output_path == "same folder":
  if creation_engine == 'ffmpeg':
   !ffmpeg -hide_banner -i "$inputFile" -vframes 1 -q:v 0 -vf "select=not(mod(n\,200)),scale=-1:480,tile=3x2" -an "$outputPath"/"$fileName"_thumbnails."$outputExtension"

if output_path == "same folder":
  if creation_engine == 'vcsi':
   !vcsi   "$inputFile" -o "$outputPath"/"$fileName"_thumbnails."$outputExtension"

if not output_path == "same folder":
   !vcsi   "$inputFile" -o "/content/drive/My Drive/Thumbnail"/"$fileName"_thumbnails."$outputExtension"


In [ ]:
#@markdown <h3>← Click Here to generate thumbnail for all video in input folder path</h3>

import os, sys, re
from IPython.display import Image, display
os.makedirs("/content/drive/My Drive/Thumbnail", exist_ok=True)

video_files_path = "" #@param {type:"string"}
output_path = 'same folder' #@param ["same folder", "My Drive/Thumbnail"]
grid = 'default' #@param ["default", "5x5", "7x7", "10x10"]

output_file_type = 'png' #@param ["png", "jpg"]
timestamp = False #@param {type:"boolean"}

if timestamp == False:
 if grid == 'default':
  if output_path == "same folder":
   !vcsi  "$video_files_path" -o "$video_files_path" -f "$output_file_type"

  if not output_path == "same folder":
   !vcsi   "$video_files_path" -o "/content/drive/My Drive/Thumbnail/" -f "$output_file_type"

 if grid == '5x5':
  if output_path == "same folder":
   !vcsi -g 5x5   "$video_files_path" -o "$video_files_path" -f "$output_file_type"

  if not output_path == "same folder":
   !vcsi -g 5x5   "$video_files_path" -o "/content/drive/My Drive/Thumbnail/" -f "$output_file_type"

 if grid == '7x7':
  if output_path == "same folder":
   !vcsi -g 7x7   "$video_files_path" -o "$video_files_path" -f "$output_file_type"

  if not output_path == "same folder":
   !vcsi -g 7x7   "$video_files_path" -o "/content/drive/My Drive/Thumbnail/" -f "$output_file_type"

 if grid == '10x10':
  if output_path == "same folder":
   !vcsi -g 10x10   "$video_files_path" -o "$video_files_path" -f "$output_file_type"

  if not output_path == "same folder":
   !vcsi -g 10x10   "$video_files_path" -o "/content/drive/My Drive/Thumbnail/" -f "$output_file_type"

else:
 if grid == 'default':
  if output_path == "same folder":
   !vcsi  -t "$video_files_path" -o "$video_files_path" -f "$output_file_type"

  if not output_path == "same folder":
   !vcsi  -t "$video_files_path" -o "/content/drive/My Drive/Thumbnail/" -f "$output_file_type"

 if grid == '5x5':
  if output_path == "same folder":
   !vcsi -t -g 5x5   "$video_files_path" -o "$video_files_path" -f "$output_file_type"

  if not output_path == "same folder":
   !vcsi -t -g 5x5   "$video_files_path" -o "/content/drive/My Drive/Thumbnail/" -f "$output_file_type"

 if grid == '7x7':
  if output_path == "same folder":
   !vcsi -t -g 7x7   "$video_files_path" -o "$video_files_path" -f "$output_file_type"

  if not output_path == "same folder":
   !vcsi -t -g 7x7   "$video_files_path" -o "/content/drive/My Drive/Thumbnail/" -f "$output_file_type"

 if grid == '10x10':
  if output_path == "same folder":
   !vcsi -t -g 10x10   "$video_files_path" -o "$video_files_path" -f "$output_file_type"

  if not output_path == "same folder":
   !vcsi -t -g 10x10   "$video_files_path" -o "/content/drive/My Drive/Thumbnail/" -f "$output_file_type"

 



##### This will create a 3x2 layout in the same directory as your video file.
* [*Example*](https://yuju.pw/y/39i2.png) *of output image*

# ✦ *Colab Still Alive Console Script:*
<p><font size=2px ><font color="red"> Tip - Set a javascript interval to click on the connect button every 60 seconds. Open developer-settings (in your web-browser) with Ctrl+Shift+I then click on console tab and type this on the console prompt. (for mac press Option+Command+I)</font></p><b>Copy script in hidden cell and paste at your browser console !!! DO NOT CLOSE YOUR BROWSER IN ORDER TO STILL RUNNING SCRIPT</b>

<code>function ClickConnect(){
console.log("Working"); 
document.querySelector("colab-connect-button").click() 
}setInterval(ClickConnect,6000)</code>

In [ ]:
#@title ← ឵឵ ឵Run this cell to crash your current Runtime if you're low on memory
#@markdown <i>After crashing, you'll have access to all the preoccupied storage</i>
some_str = ' ' * 5120000000000